### Prerequisite

- Create an Azure Custom Vision Resource
    - (technically, you are creating two resources. One for training and one for prediction)
- Create a Custom Vision project in the Custom Vision portal
    - You could do this in code via the SDK: [Link Here](https://learn.microsoft.com/en-us/azure/ai-services/custom-vision-service/quickstarts/image-classification?tabs=windows%2Cvisual-studio&pivots=programming-language-python#install-the-client-library)
- Upload and tag images (supervised learning)
    - Similarly, you could also do this via code (refer to the link above)
- Train the model
- Look at the model's performance and test the model with some images you find online

- Create your `.env` file

#### Note: View the project settings <br/>
The project you have created has been assigned a unique identifier, which you will need to specify in any code that interacts with it.
1. Click the settings (⚙) icon at the top right of the Performance page to view the project settings.
2. Under General (on the left), note the Project Id that uniquely identifies this project.
3. On the right, under Resources note that the key and endpoint are shown. These are the details for the training resource (you can also obtain this information by viewing the resource in the Azure portal).

### Use the training API

The Custom Vision portal provides a convenient user interface that you can use to upload and tag images, and train models. However, in some scenarios you may want to automate model training by using the Custom Vision training API.

In [ ]:
!pip install azure-cognitiveservices-vision-customvision

In [ ]:
#Install necessary libraries -- following guide on quickstart
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

#and for our env file
from dotenv import load_dotenv

In [10]:
#load env
load_dotenv()

True

In [15]:
# retrieve environment variables
training_endpoint = os.environ["CUSTOM_IMAGE_TRAIN_ENDPOINT"]
training_key = os.environ["CUSTOM_IMAGE_TRAIN_KEY"]
prediction_key = os.environ["CUSTOM_IMAGE_PREDICT_KEY"]
project_id = '84fd0da0-7218-4990-8d0a-7c60936d9c2d'

In [16]:
# Authenticate a client for the training API
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
training_client = CustomVisionTrainingClient(training_endpoint, credentials)

In [17]:
# Get the Custom Vision project
custom_vision_project = training_client.get_project(project_id)

Now We need to upload and tag images

In [21]:
def Upload_Images(folder):
    tags = training_client.get_tags(custom_vision_project.id) #gets all of the defined tags
    for tag in tags: #looping through the tags we defined in the studio
        print(tag.name) #printing just to show
        for image in os.listdir(os.path.join(folder,tag.name)): #for each file in the assorted directories
            image_data = open(os.path.join(folder,tag.name,image), "rb").read() #serialize that file
            training_client.create_images_from_data(custom_vision_project.id, image_data, [tag.id])
        

In [ ]:
# Upload and tag images
Upload_Images('images/more-training-images')

'''
To make this really robust regardless of OS, we could do:
base_folder = os.path.abspath("images/more-training-images")
Upload_Images(base_folder)
'''

banana
orange
apple


### Use The Prediction API

Make sure the model is published


In [26]:
#Import libraries
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient


In [27]:
prediction_endpoint = os.getenv('CUSTOM_IMAGE_PREDICT_ENDPOINT')
prediction_key = os.getenv('CUSTOM_IMAGE_PREDICT_KEY')
project_id = '84fd0da0-7218-4990-8d0a-7c60936d9c2d' #repeat from above
model_name = 'fruit-classifier-test'

In [28]:
# Authenticate a client for the prediction API
credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
prediction_client = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

Test one image

In [30]:
os.listdir('images/test-images')

['IMG_TEST_1.jpg', 'IMG_TEST_2.jpg', 'IMG_TEST_3.jpg']

In [31]:
image_data = open(os.path.join('images/test-images', 'IMG_TEST_1.jpg'), "rb").read()

In [34]:
results = prediction_client.classify_image(project_id, model_name, image_data)

In [35]:
results

In [37]:
results.as_dict()

{'id': '8283daef-0734-4ed8-b17e-d5b222c99fb0',
 'project': '84fd0da0-7218-4990-8d0a-7c60936d9c2d',
 'iteration': 'f0adcc0d-2410-468b-85ed-fe03c66c035b',
 'created': '2025-06-26T19:19:13.944Z',
 'predictions': [{'probability': 0.9816808,
   'tag_id': '0e600077-017b-40f4-9182-98df9d3fc339',
   'tag_name': 'apple',
   'tag_type': 'Regular'},
  {'probability': 0.01828001,
   'tag_id': '61f3810b-21cb-4129-b05c-81aa0ce3a327',
   'tag_name': 'orange',
   'tag_type': 'Regular'},
  {'probability': 3.9220165e-05,
   'tag_id': '4d8a6c71-96bd-43d1-bb61-235bc34648d1',
   'tag_name': 'banana',
   'tag_type': 'Regular'}]}

In [44]:
results.predictions[0].probability

0.9816808

Let's now add some code to loop through all of our test images

In [45]:
for image in os.listdir('images/test-images'): #for each image we want to test
            image_data = open(os.path.join('images/test-images',image), "rb").read() #open that image
            results = prediction_client.classify_image(project_id, model_name, image_data) #classify that image

            # Loop over each label prediction and print any with probability > 50%
            for prediction in results.predictions:
                if prediction.probability > 0.5:
                    print(image, ': {} ({:.0%})'.format(prediction.tag_name, prediction.probability))

IMG_TEST_1.jpg : apple (98%)
IMG_TEST_2.jpg : banana (100%)
IMG_TEST_3.jpg : orange (100%)
